In [3]:
import pandas as pd
from sklearn.utils import shuffle

train = pd.read_csv("C:/Users/Valentina/Desktop/UDSC/Sentiment analysis/Data/Amazon/train.csv", sep="|")
data_Amazon = pd.read_csv("C:/Users/Valentina/Desktop/UDSC/Sentiment analysis/Data/Amazon/test.csv", sep="|")
data_MR = pd.read_csv("C:/Users/Valentina/Desktop/UDSC/Sentiment analysis/Data/Amazon/movie reviews.csv", sep="|")
train = train.rename(columns={'overall' : 'label', 'reviewText' : 'text'})
data_Amazon = data_Amazon.rename(columns={'overall' : 'label', 'reviewText' : 'text'})
train = train.astype(str)
data_Amazon = data_Amazon.astype(str)
data_MR = data_MR.astype(str)

In [4]:
print(train.shape)
train.groupby('label').count()

(1343971, 2)


,text
label,
0,185410
1,1158561


In [5]:
import re, nltk
from nltk.stem import SnowballStemmer
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    word_list = nltk.word_tokenize(text)
    stemmer = SnowballStemmer("english")
    stems = [stemmer.stem(word) for word in word_list]
    return stems

In [ ]:
%%time

from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, stop_words = 'english')

data_features = vectorizer.fit_transform(train['text'])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        train.text, 
        train.label,
        test_size=0.2, 
        random_state=42)


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('vectorizer', HashingVectorizer(tokenizer=tokenize, ngram_range=(1, 3),
                              analyzer = 'word', binary = True, stop_words = 'english')), 
                     ('tfidf', TfidfTransformer()),
                     ('classifier', LogisticRegression(C = 100))])
model = pipeline.fit(X=X_train, y=y_train)

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print ("Validation Accuracy Amazon :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

X_test_MR, y_test_MR = (data_MR.text, data_MR.label)
y_pred_MR = model.predict(X_test_MR)

print ("Test Accuracy MR :", accuracy_score(y_test_MR, y_pred_MR))
print(classification_report(y_test_MR, y_pred_MR))

X_test_amazon, y_test_amazon = (data_Amazon.text, data_Amazon.label)
y_pred_amazon = model.predict(X_test_amazon)

print ("Test Accuracy Amazon :", accuracy_score(y_test_amazon, y_pred_amazon))
print(classification_report(y_test_amazon, y_pred_amazon))